![i2b2 Logo](images/transmart-logo.png)

# Using ChromaDB + Embeddings to Search Patient Notes (RAG)

This notebook demonstrates how to implement a **Retrieval-Augmented Generation (RAG)** pipeline using **local embeddings** and **ChromaDB** to search and analyze clinical notes stored in an i2b2-like format. It walks through decoding, embedding, storing, retrieving, and generating responses with a local LLM.

You will work through a complete pipeline:
- Decode notes from BinHex format
- Embed clinical notes using the MiniLM model
- Store the notes and metadata in a persistent ChromaDB vector store
- Perform semantic similarity search and inspect cosine-based scores
- Apply **Maximal Marginal Relevance (MMR)** to promote diversity and reduce redundancy
- Retrieve only the **most recent note per patient** for clarity and accuracy
- Inject results into a reusable prompt template
- Generate structured, clinically meaningful responses using a local LLM (e.g., Qwen or LLaMA 3 via Ollama)

### 🔍 Key Concepts Covered

- Decoding i2b2-formatted BinHex clinical notes
- Embedding full notes for semantic similarity search
- Persistent storage and retrieval using ChromaDB
- Score-based filtering and MMR ranking
- Structured prompt engineering for LLMs
- Zero-shot summarization with a local model

Each cell builds on the last to demonstrate a complete RAG workflow, tailored to **clinical informatics and patient note analysis**.

> This notebook is part of the workshop: _Using LLMs to Search and Summarize Patient Notes_.


## 1. Prepare Data for Embedding

Before we can perform semantic search or generate responses using clinical notes, we must first prepare the data.

- **1.1**: Load a simulated i2b2 `visit_dimension` table that contains BinHex-encoded clinical notes and decode them into readable plain text.

This step ensures that each clinical note is in a usable format for embedding in the next section using a local MiniLM model.


In [34]:
# -----------------------------------------------------------
# 1.1. Load and Decode Clinical Notes from i2b2-Mimicking CSV
# -----------------------------------------------------------
# This cell loads visit-level data from a CSV file that mimics the i2b2 `visit_dimension` table
# and decodes the BinHex-encoded clinical notes into readable plain text.

# Each record includes:
# - encounter_num: Unique visit ID
# - patient_num: Patient identifier
# - start_date, end_date: Visit dates
# - location_cd, location_path: Clinic metadata
# - visit_blob: BinHex-encoded clinical note text
# - note_text: Decoded BinHex clinical note text

import pandas as pd
import binascii
from IPython.display import display, Markdown

# Load the i2b2-style data
csv_path = "datafiles/i2b2_encounter_table.csv"
df = pd.read_csv(csv_path)

# Function to decode BinHex-encoded notes
def decode_note(hex_blob):
    hex_str = hex_blob.replace("0x", "")
    return binascii.unhexlify(hex_str).decode("utf-8", errors="ignore")

# Decode all notes into a new column
df["note_text"] = df["visit_blob"].apply(decode_note)

# Preview the first 10 records with decoded notes
display(Markdown("### Preview of Decoded Notes"))
display(df.head(10))


### Preview of Decoded Notes

,encounter_num,patient_num,start_date,end_date,inout_cd,location_cd,location_path,visit_blob,note_text
0,475303,1000000001,01/16/2003,01/16/2003,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n - Patient ID: 10000...
1,479681,1000000001,03/29/2007,03/29/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
2,480315,1000000001,09/20/2007,09/20/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n\n - Patient ID: 10...
3,480903,1000000001,03/04/2008,03/04/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x5468697320697320612032332D796561722D6F6C6420...,This is a 23-year-old Black female who has bee...
4,481398,1000000001,08/11/2008,08/11/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A20202020...,Visit Information: \n - Patient ID: 1000...
5,482655,1000000001,05/18/2009,05/18/2009,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A0A202020...,Visit Information:\n - Patient ID: 10000000...
6,471658,1000000002,04/17/1998,04/17/1998,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
7,472076,1000000002,01/04/1999,01/04/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
8,472473,1000000002,08/12/1999,08/12/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...
9,472692,1000000002,12/01/1999,12/01/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...,**Visit Information:**\n- Patient ID: 10000000...


## 2. Decode, Embed, and Store Entire Clinical Notes in ChromaDB

In this step, we process and store each clinical note as a **complete document** in a ChromaDB vector store. This approach preserves full patient context, making it ideal for semantic search and downstream clinical reasoning.

### Step 2.1: Embed and Store Notes
1. **Embed Full Notes**
   Each note is converted into a semantic vector using a lightweight transformer model (`MiniLM`).

2. **Store in ChromaDB**
   The embedded vector and its associated metadata (e.g., patient ID, encounter number, visit date) are stored together in a persistent ChromaDB directory.

### Why This Approach?

Storing full notes is especially useful when:
- You want to retrieve the complete narrative for clinical context
- Your downstream task (e.g., summarization or decision support) depends on comprehensive input
- Each note fits within the input limit of a single LLM call

This setup supports more faithful summarization and reasoning than chunk-based approaches when notes are relatively short and self-contained.

<img src="./images/rag_full.png" alt="RAG Full" width="900">


In [36]:
# -----------------------------------------------------------
# 2.1. Embed Clinical Notes Using Local MiniLM Embeddings and Store in ChromaDB
# -----------------------------------------------------------
# This cell encodes each clinical note into a semantic vector using a local MiniLM model
# and stores the results along with metadata in a ChromaDB vector store for retrieval.

# Embedding Model: sentence-transformers/all-MiniLM-L6-v2
# - Optimized for fast local execution
# - Produces 384-dimensional vectors for semantic similarity

# Required packages:
#   pip install langchain langchain-huggingface chromadb

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create or connect to the persistent ChromaDB directory
vectorstore = Chroma(
    persist_directory="./datafiles/chroma_db_notes",
    embedding_function=embedding_model
)

# Prepare documents and metadata
documents = df["note_text"].tolist()
metadata = df[["patient_num", "encounter_num", "start_date"]].to_dict(orient="records")

# Add text and metadata to the ChromaDB vector store
vectorstore.add_texts(texts=documents, metadatas=metadata)

print(f"✅ Successfully embedded and stored {len(documents)} clinical notes using MiniLM and ChromaDB.")


✅ Successfully embedded and stored 1128 clinical notes using MiniLM and ChromaDB.


## 3. Retrieving Clinical Notes with Similarity Score (RAG Retrieval with ChromaDB)

In this section, we retrieve relevant clinical notes from a ChromaDB vector store using vector similarity techniques. We explore both standard and advanced retrieval strategies to improve the relevance and diversity of retrieved context for downstream LLM generation.

### Key Retrieval Strategies

1. **Define a Clinical Query**
   - The user provides a natural language question (e.g., "Who has asthma and is taking Fluticasone and Albuterol?").

2. **Similarity Search with Scores (Step 3.2)**
   - Retrieves the top-k notes ranked by semantic similarity to the query.
   - Returns cosine-based relevance scores for each match.

3. **Score Threshold Filtering (Step 3.3)**
   - Filters out low-confidence matches using a minimum similarity score.
   - Retains only notes that meet a defined relevance threshold (e.g., ≥ 0.6).

4. **Maximal Marginal Relevance (MMR) Search (Step 3.4)**
   - Balances similarity and diversity.
   - Reduces redundancy by selecting a diverse subset of highly relevant notes.

### Why Use These Strategies?

High-quality retrieval is critical to the success of RAG workflows. These techniques help:
- Improve the contextual relevance of inputs to the LLM
- Filter out irrelevant or low-confidence documents
- Encourage diverse results to reduce bias and improve robustness

<img src="./images/rag_retrieval.png" alt="RAG Retrieval" width="900">


In [37]:
# -----------------------------------------------------------
# 3.1. Define the Query for Clinical Note Retrieval
# -----------------------------------------------------------
# This cell defines a natural language query to search the embedded clinical notes
# stored in ChromaDB using semantic similarity.

# Concept:
# The query will be embedded and compared against the stored note vectors
# to find the most semantically relevant matches.

# Example Clinical Query:
query = "Who has asthma and is taking Fluticasone and Albuterol?"

# Display the query for reference
from IPython.display import Markdown, display
display(Markdown(f"### 🔍 Query: `{query}`"))


### 🔍 Query: `Who has asthma and is taking Fluticasone and Albuterol?`

In [39]:
# -----------------------------------------------------------
# 3.2. Perform Similarity Search with Relevance Scores
# -----------------------------------------------------------
# This cell retrieves clinical notes most semantically similar to the input query.
# Each result includes a relevance score that indicates how well the note matches the query.

# Function used:
# - vectorstore.similarity_search_with_relevance_scores(query, k=10)
#   Returns the top-k notes with similarity scores (cosine-based).

# Relevance Score Interpretation:
# - 0.90 – 1.00 → Highly relevant
# - 0.70 – 0.90 → Strong relevance
# - 0.50 – 0.70 → Moderate relevance
# - 0.30 – 0.50 → Low relevance
# - 0.00 – 0.30 → Minimal or no relevance

from IPython.display import display, Markdown

# Perform the similarity search
results = vectorstore.similarity_search_with_relevance_scores(query, k=10)

# Display the results
display(Markdown("### 🔍 Retrieved Clinical Notes with Relevance Scores"))

for idx, (doc, score) in enumerate(results, 1):
    patient = doc.metadata.get("patient_num", "N/A")
    encounter = doc.metadata.get("encounter_num", "N/A")
    date = doc.metadata.get("start_date", "N/A")
    doc_id = doc.id
    excerpt = doc.page_content[:1000].replace("\n", " ")

    display(Markdown(
        f"---\n**Result {idx}**  \n"
        f"- **Relevance Score:** `{score:.4f}`  \n"
        f"- **Patient Num:** `{patient}`  \n"
        f"- **Encounter Num:** `{encounter}`  \n"
        f"- **Start Date:** `{date}`  \n"
        f"- **Document ID:** `{doc_id}`  \n\n"
        f"**Excerpt:**\n```text\n{excerpt}...\n```"
    ))


### 🔍 Retrieved Clinical Notes with Relevance Scores

---
**Result 1**  
- **Relevance Score:** `0.6289`  
- **Patient Num:** `1000000005`  
- **Encounter Num:** `477663`  
- **Start Date:** `06/21/2005`  
- **Document ID:** `fd7856a3-a4dc-476e-85bd-daac63bcc2f7`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000005 - Encounter ID: 477663 - Visit Date: June 21, 2005  **Subjective:**  This is a 32-year-old Hispanic female who has been receiving care at our clinic for approximately 3 months. She speaks English and presents today for a follow-up visit scheduled as part of her ongoing asthma management. The patient reports continued asthma symptoms, including persistent shortness of breath, wheezing particularly at night, and a cough that disrupts her sleep. Over the past two weeks, these symptoms have intensified despite regular usage of her asthma medications: albuterol inhaler for rescue, daily fluticasone inhaler, and nightly montelukast. She states the albuterol provides only brief relief.  She has a medical history of unspecified asthma without mention of status asthmaticus, back sprain from unspecified causes, vaginitis, and a previous high-risk pregnancy requiring special investigations. Her social history includes living in an urban environment ...
```

---
**Result 2**  
- **Relevance Score:** `0.6143`  
- **Patient Num:** `1000000011`  
- **Encounter Num:** `476139`  
- **Start Date:** `11/20/2003`  
- **Document ID:** `533d70d1-c60a-4d28-9501-633f4ad9aa81`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000011 - Encounter ID: 476139 - Visit Date: Nov 20, 2003  **Subjective:** This is a 54-year-old Caucasian female who speaks English and has been receiving care at our clinic. She presents today for a follow-up regarding her recurrent asthma and associated symptoms. The patient is experiencing persistent shortness of breath, wheezing, and a nocturnal cough that interrupts her sleep. These symptoms have been particularly troublesome over the last month despite adherence to her current medication regimen. She denies smoking and has reported no recent exposure to known allergens or new environmental triggers.  Her past medical history is significant for recurrent asthma, an acute myocardial infarction, hypertension, hypercholesterolemia, lumbar disc displacement leading to chronic lumbago, cervical dysplasia, and a panic disorder. Her medications include fluticasone and an albuterol inhaler for asthma, along with antihypertensive and lipid-lowering ...
```

---
**Result 3**  
- **Relevance Score:** `0.6027`  
- **Patient Num:** `1000000123`  
- **Encounter Num:** `475208`  
- **Start Date:** `11/27/2002`  
- **Document ID:** `02c925e8-38cd-4aa2-8058-f3e13d49e2a2`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000123 - Encounter ID: 475208 - Visit Date: November 27, 2002  **Subjective:** This is a 19-year-old Indian male who has been under our care for asthma management. The patient speaks German and came to the clinic on November 27, 2002, for a follow-up visit. Since his last appointment in March 2001, he has experienced an uptick in both the frequency and severity of his asthma attacks. Recently, his nocturnal asthma episodes have become more disruptive, leading to sleep disturbances and affecting his daily function and academic performance. The patient reports consistent symptoms of wheezing, shortness of breath, and chest tightness that are exacerbated by physical activities and cold weather. He has been reliant on his rescue inhaler, using it three to four times a day with only partial relief.  His medical history includes chronic asthma, which has persisted since childhood. He is currently managed with a fluticasone/salmeterol inhaler taken twi...
```

---
**Result 4**  
- **Relevance Score:** `0.6026`  
- **Patient Num:** `1000000005`  
- **Encounter Num:** `475726`  
- **Start Date:** `07/02/2003`  
- **Document ID:** `4448fa26-08b7-4494-9ddc-45472e13a690`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000005 - Encounter ID: 475726 - Visit Date: July 2, 2003  **Subjective:**  This is a 30-year-old Hispanic female who is an English speaker, followed at our clinic for asthma management. Today, she presents for a routine follow-up visit. The patient reports persistent symptoms of asthma, including increased shortness of breath, wheezing, and a nocturnal cough that disrupts her sleep. She mentions that these symptoms have worsened over the past two weeks despite regular use of her asthma medications. She is diligent with her albuterol inhaler (used as needed), fluticasone inhaler (daily), and montelukast (nightly), yet finds that the albuterol offers only temporary relief.  Her medical history includes asthma with multiple past exacerbations usually triggered by allergens or respiratory infections. There are no past surgeries of note. Socially, she resides in an urban environment with high allergen exposure and works as a primary school teacher. S...
```

...

---
**Result 6**  
- **Relevance Score:** `0.5955`  
- **Patient Num:** `1000000123`  
- **Encounter Num:** `471535`  
- **Start Date:** `01/05/1998`  
- **Document ID:** `d5aea114-7143-4d95-b11e-bab963154fde`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000123 - Encounter ID: 471535 - Visit Date: January 5, 1998  **Subjective:** This is a 15-year-old Indian male who has been a patient at our clinic, presenting today for a follow-up visit. He speaks German and has a well-documented history of asthma. The patient reports experiencing increased frequency and severity of asthma attacks over the past month, particularly at night, resulting in disturbed sleep and difficulty concentrating at school. He describes his symptoms as wheezing, shortness of breath, and chest tightness, often triggered by exercise and cold air. He has been using his rescue inhaler more frequently, approximately three to four times daily, which provides temporary relief.   Past medical history is significant for persistent asthma since early childhood, managed with inhaled corticosteroids and a rescue inhaler. He has not undergone any surgeries. Currently, he is taking fluticasone/salmeterol inhaler twice daily and albuterol a...
```

---
**Result 7**  
- **Relevance Score:** `0.5952`  
- **Patient Num:** `1000000005`  
- **Encounter Num:** `481497`  
- **Start Date:** `09/02/2008`  
- **Document ID:** `2936aabe-7401-4bc0-96e0-0af231f1ae54`  

**Excerpt:**
```text
Visit Information:    - Patient ID: 1000000005    - Encounter ID: 481497    - Visit Date: 09/02/2008  This is a 35-year-old Hispanic female who has been a patient at our clinic for over a year, presenting today for follow-up on her chronic asthma and routine health examination. The patient speaks English well. She reports her asthma symptoms have worsened over the past two weeks, including increased shortness of breath, constant wheezing, and persistent cough. Despite regular use of an albuterol inhaler, daily fluticasone (an inhaled corticosteroid), and nightly montelukast, her symptom control remains poor with only short-term relief from albuterol.  She has a history of episodic asthma without status asthmaticus, sarcoidosis, pain in limbs, and disturbance of skin sensation. There is no history of smoking or alcohol use. She lives in an urban area with high allergen exposure and works as a primary school teacher, which might contribute to her symptoms.  During the physical examinatio...
```

---
**Result 8**  
- **Relevance Score:** `0.5939`  
- **Patient Num:** `1000000005`  
- **Encounter Num:** `479401`  
- **Start Date:** `12/29/2006`  
- **Document ID:** `a6a25b6a-54c4-4bdf-a75d-45f6131983bc`  

**Excerpt:**
```text
 **Visit Information:** - Patient ID: 1000000005 - Encounter ID: 479401 - Visit Date: Dec 29, 2006  **Subjective:**  This is a 33-year-old Hispanic female who has been receiving care at our clinic over several visits. She speaks English and presented today for a follow-up visit to manage her asthma. She reports that her asthma symptoms, particularly shortness of breath, wheezing, and a persistent cough, have been worsening over the past two weeks despite adhering to her medication regimen which includes an albuterol inhaler as needed, a daily inhaled corticosteroid (fluticasone), and nightly montelukast. She notes that while the albuterol offers brief symptom relief, it has been insufficient to manage her exacerbations effectively.  Her medical history includes chronic asthma without status asthmaticus, non-specific epigastric and upper left quadrant abdominal pain, irritable bowel syndrome, painful respiration, and periodic vaginal infections. She is a non-smoker and non-drinker, livi...
```

---
**Result 9**  
- **Relevance Score:** `0.5904`  
- **Patient Num:** `1000000058`  
- **Encounter Num:** `471856`  
- **Start Date:** `09/08/1998`  
- **Document ID:** `f82e2a89-52bf-4e2e-a031-a2df1a9abea6`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000058 - Encounter ID: 471856 - Visit Date: 09/08/1998  This is a 7-year-old Hispanic male who has been receiving care at our clinic for asthma and related conditions since the age of three. He returns today for a follow-up appointment accompanied by his mother, whose primary language is Spanish. They are here to discuss the management of his asthma and associated respiratory issues.  **Subjective:** The patient’s mother reports an increased frequency of coughing and wheezing over the past month, particularly disrupting sleep at night and occurring after physical activity. There have been no recent episodes of fever or respiratory infections. The patient has been using his albuterol inhaler approximately three times per week.  The patient’s past medical history is significant for asthma and seasonal allergies, with several years of documentation. His surgical history includes a unilateral repair of an indirect inguinal hernia and excision of a h...
```

---
**Result 10**  
- **Relevance Score:** `0.5899`  
- **Patient Num:** `1000000024`  
- **Encounter Num:** `482121`  
- **Start Date:** `01/02/2009`  
- **Document ID:** `291bbef6-1564-4605-9e78-a0faddd92979`  

**Excerpt:**
```text
**Visit Information:** - **Patient ID:** 1000000024 - **Encounter ID:** 482121 - **Visit Date:** January 2, 2009  **Subjective:** This is a 30-year-old Black male who has been receiving care at our clinic for approximately 3 months. He speaks English and presents today for a follow-up of his chronic asthma, which is the primary reason for his visit. He mentions intermittent symptoms of wheezing and shortness of breath, particularly at night and when engaging in physical activity. His asthma symptoms typically respond to his prescribed inhaled corticosteroid (fluticasone) and rescue inhaler (albuterol). However, he experiences symptomatic episodes roughly twice a week. The patient denies any recent respiratory infections or emergency department visits related to asthma exacerbation.   Additionally, he reports a persistent rash that first appeared two weeks ago. He does not attribute this to any known allergens or recent changes in his environment. He notes some mild itching but no signi...
```

In [40]:
# -----------------------------------------------------------
# 3.3. Use a Retriever with a Similarity Score Threshold
# -----------------------------------------------------------
# This cell configures a retriever that only returns documents whose
# similarity scores exceed a predefined threshold.

# Parameters:
# - search_type="similarity_score_threshold": Enables threshold-based filtering.
# - search_kwargs={"k": 10, "score_threshold": 0.6}
#     - k: Max number of documents to evaluate
#     - score_threshold: Minimum similarity score required for inclusion

# Purpose:
# This improves retrieval precision by excluding weak matches—essential
# for clinical reasoning and safety-critical applications.

from IPython.display import display, Markdown

# Define threshold and top-k limit
score_threshold = 0.6
top_k = 10

# Create the retriever with filtering
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": top_k, "score_threshold": score_threshold}
)

# Execute the retrieval
results = retriever.invoke(query)

# Display results
display(Markdown(f"### 🔎 Retrieved Clinical Notes (Score ≥ {score_threshold})"))

for idx, doc in enumerate(results, 1):
    patient = doc.metadata.get("patient_num", "N/A")
    date = doc.metadata.get("start_date", "N/A")
    doc_id = doc.id
    excerpt = doc.page_content[:1000].replace("\n", " ")

    display(Markdown(
        f"**Document {idx}**  \n"
        f"- **Patient Num:** `{patient}`  \n"
        f"- **Start Date:** `{date}`  \n"
        f"- **Document ID:** `{doc_id}`  \n\n"
        f"**Excerpt:**\n```text\n{excerpt}...\n```"
    ))

display(Markdown(f"**✅ Total relevant results:** `{len(results)}`"))


### 🔎 Retrieved Clinical Notes (Score ≥ 0.6)

**Document 1**  
- **Patient Num:** `1000000005`  
- **Start Date:** `06/21/2005`  
- **Document ID:** `fd7856a3-a4dc-476e-85bd-daac63bcc2f7`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000005 - Encounter ID: 477663 - Visit Date: June 21, 2005  **Subjective:**  This is a 32-year-old Hispanic female who has been receiving care at our clinic for approximately 3 months. She speaks English and presents today for a follow-up visit scheduled as part of her ongoing asthma management. The patient reports continued asthma symptoms, including persistent shortness of breath, wheezing particularly at night, and a cough that disrupts her sleep. Over the past two weeks, these symptoms have intensified despite regular usage of her asthma medications: albuterol inhaler for rescue, daily fluticasone inhaler, and nightly montelukast. She states the albuterol provides only brief relief.  She has a medical history of unspecified asthma without mention of status asthmaticus, back sprain from unspecified causes, vaginitis, and a previous high-risk pregnancy requiring special investigations. Her social history includes living in an urban environment ...
```

**Document 2**  
- **Patient Num:** `1000000011`  
- **Start Date:** `11/20/2003`  
- **Document ID:** `533d70d1-c60a-4d28-9501-633f4ad9aa81`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000011 - Encounter ID: 476139 - Visit Date: Nov 20, 2003  **Subjective:** This is a 54-year-old Caucasian female who speaks English and has been receiving care at our clinic. She presents today for a follow-up regarding her recurrent asthma and associated symptoms. The patient is experiencing persistent shortness of breath, wheezing, and a nocturnal cough that interrupts her sleep. These symptoms have been particularly troublesome over the last month despite adherence to her current medication regimen. She denies smoking and has reported no recent exposure to known allergens or new environmental triggers.  Her past medical history is significant for recurrent asthma, an acute myocardial infarction, hypertension, hypercholesterolemia, lumbar disc displacement leading to chronic lumbago, cervical dysplasia, and a panic disorder. Her medications include fluticasone and an albuterol inhaler for asthma, along with antihypertensive and lipid-lowering ...
```

**Document 3**  
- **Patient Num:** `1000000123`  
- **Start Date:** `11/27/2002`  
- **Document ID:** `02c925e8-38cd-4aa2-8058-f3e13d49e2a2`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000123 - Encounter ID: 475208 - Visit Date: November 27, 2002  **Subjective:** This is a 19-year-old Indian male who has been under our care for asthma management. The patient speaks German and came to the clinic on November 27, 2002, for a follow-up visit. Since his last appointment in March 2001, he has experienced an uptick in both the frequency and severity of his asthma attacks. Recently, his nocturnal asthma episodes have become more disruptive, leading to sleep disturbances and affecting his daily function and academic performance. The patient reports consistent symptoms of wheezing, shortness of breath, and chest tightness that are exacerbated by physical activities and cold weather. He has been reliant on his rescue inhaler, using it three to four times a day with only partial relief.  His medical history includes chronic asthma, which has persisted since childhood. He is currently managed with a fluticasone/salmeterol inhaler taken twi...
```

**Document 4**  
- **Patient Num:** `1000000005`  
- **Start Date:** `07/02/2003`  
- **Document ID:** `4448fa26-08b7-4494-9ddc-45472e13a690`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000005 - Encounter ID: 475726 - Visit Date: July 2, 2003  **Subjective:**  This is a 30-year-old Hispanic female who is an English speaker, followed at our clinic for asthma management. Today, she presents for a routine follow-up visit. The patient reports persistent symptoms of asthma, including increased shortness of breath, wheezing, and a nocturnal cough that disrupts her sleep. She mentions that these symptoms have worsened over the past two weeks despite regular use of her asthma medications. She is diligent with her albuterol inhaler (used as needed), fluticasone inhaler (daily), and montelukast (nightly), yet finds that the albuterol offers only temporary relief.  Her medical history includes asthma with multiple past exacerbations usually triggered by allergens or respiratory infections. There are no past surgeries of note. Socially, she resides in an urban environment with high allergen exposure and works as a primary school teacher. S...
```

**✅ Total relevant results:** `4`

In [42]:
# -----------------------------------------------------------
# 3.4. Perform Maximal Marginal Relevance (MMR) Search
# -----------------------------------------------------------
# This cell retrieves clinical notes using Maximal Marginal Relevance (MMR),
# which balances relevance to the query with diversity across the results.

# Parameters:
# - fetch_k = 500: Number of top-ranked candidates to evaluate
# - k = 5: Final number of diverse, relevant documents to return
# - lambda_mult:
#     - 0.0 → maximize diversity
#     - 1.0 → maximize relevance
#     - 0.5 → balance between both

# Purpose:
# MMR reduces redundancy by penalizing near-duplicates while still prioritizing
# clinical notes that are relevant to the query. This is helpful when multiple
# perspectives or variations are valuable (e.g., different asthma treatment plans).

from IPython.display import display, Markdown

# Perform MMR search
results = vectorstore.max_marginal_relevance_search(
    query=query,
    k=5,
    fetch_k=500,
    lambda_mult=0.5
)

# Display results
display(Markdown("### 📘 Retrieved Clinical Notes Using Maximal Marginal Relevance (MMR)"))

for idx, doc in enumerate(results, 1):
    patient = doc.metadata.get("patient_num", "N/A")
    date = doc.metadata.get("start_date", "N/A")
    doc_id = getattr(doc, "id", "N/A")
    excerpt = doc.page_content[:1000].replace("\n", " ")

    display(Markdown(
        f"**Document {idx}**  \n"
        f"- **Patient Num:** `{patient}`  \n"
        f"- **Start Date:** `{date}`  \n"
        f"- **Document ID:** `{doc_id}`  \n\n"
        f"**Excerpt:**\n```text\n{excerpt}...\n```"
    ))

display(Markdown(f"**✅ Total MMR Results Returned:** `{len(results)}`"))


### 📘 Retrieved Clinical Notes Using Maximal Marginal Relevance (MMR)

**Document 1**  
- **Patient Num:** `1000000005`  
- **Start Date:** `06/21/2005`  
- **Document ID:** `fd7856a3-a4dc-476e-85bd-daac63bcc2f7`  

**Excerpt:**
```text
**Visit Information:** - Patient ID: 1000000005 - Encounter ID: 477663 - Visit Date: June 21, 2005  **Subjective:**  This is a 32-year-old Hispanic female who has been receiving care at our clinic for approximately 3 months. She speaks English and presents today for a follow-up visit scheduled as part of her ongoing asthma management. The patient reports continued asthma symptoms, including persistent shortness of breath, wheezing particularly at night, and a cough that disrupts her sleep. Over the past two weeks, these symptoms have intensified despite regular usage of her asthma medications: albuterol inhaler for rescue, daily fluticasone inhaler, and nightly montelukast. She states the albuterol provides only brief relief.  She has a medical history of unspecified asthma without mention of status asthmaticus, back sprain from unspecified causes, vaginitis, and a previous high-risk pregnancy requiring special investigations. Her social history includes living in an urban environment ...
```

**Document 2**  
- **Patient Num:** `1000000088`  
- **Start Date:** `10/28/2004`  
- **Document ID:** `42f7694c-98b4-46f8-9acc-7b9523213d0e`  

**Excerpt:**
```text
Visit Information: - Patient ID: 1000000088 - Encounter ID: 477031 - Visit Date: Oct 28 2004  This is a 9-year-old Asian male, a German speaker, who has been receiving care at our clinic for the past two years. He presents today for a follow-up visit primarily to address complications associated with his asthma.  Subjective: The young patient and his mother report episodes of increased wheezing and shortness of breath occurring predominantly at night. The mother notes that the use of the albuterol inhaler has significantly increased, and he experiences more frequent coughing and chest tightness, particularly with physical activity or exposure to allergens like pollen and dust. He has an established history of asthma and allergic rhinitis. Moreover, his medical history is significant for cystic fibrosis, postinflammatory pulmonary fibrosis, and concerns regarding his growth and nutritional status. Current medications include a daily inhaled corticosteroid for asthma, albuterol for acute...
```

**Document 3**  
- **Patient Num:** `1000000105`  
- **Start Date:** `02/04/2009`  
- **Document ID:** `539ebc57-152c-4a84-baa3-3d7b9c6e4e63`  

**Excerpt:**
```text
This is a 65-year-old Hispanic male who has been receiving care at our clinic for the past few months. He speaks German and presented today, February 4, 2009, for a follow-up concerning his asthma and overall health management. The patient has a primary diagnosis of unspecified asthma, experiencing increased asthma attacks over the past two weeks, particularly in the early mornings and late evenings. These episodes are characterized by significant shortness of breath and wheezing. The patient has been using his rescue inhaler, Albuterol, 5-6 times per day but denies any nocturnal symptoms affecting his sleep.  The patient has a comprehensive medical history, which includes a chronic obstructive form of asthma, a history of malignant neoplasm of the prostate, previously treated with radical prostatectomy, and hypertension. Other notable conditions include a history of tobacco use disorder and prior preoperative cardiovascular examinations. He also underwent a simple excision of a lympha...
```

**Document 4**  
- **Patient Num:** `1000000112`  
- **Start Date:** `06/10/2005`  
- **Document ID:** `334ce254-4040-4ca6-8d59-9e7793d50d73`  

**Excerpt:**
```text
Visit Information: - Patient ID: 1000000112 - Encounter ID: 477645 - Visit Date: 06/10/2005  This is a 12-year-old Black male who has been receiving care at our clinic for approximately three years. He is Spanish-speaking and presented today, June 10, 2005, for a follow-up visit concerning his asthma management.  Subjective: The primary concerns today remain focused on the patient’s chronic asthma. Over the past week, he has been experiencing increased symptoms, including wheezing, dyspnea, particularly during physical activity, and nocturnal cough, which have necessitated an increased use of his Atrovent inhaler. The patient denies any associated fever or chest pain. Despite adherence to his current medication regimen, these symptoms have persisted, indicating suboptimal asthma control.  The patient’s medical history is significant for multiple diagnoses of asthma, with subtypes including acute exacerbations and status asthmaticus, as well as past pneumonia and pulmonary collapse. The...
```

**Document 5**  
- **Patient Num:** `1000000110`  
- **Start Date:** `11/13/2008`  
- **Document ID:** `f8e48e4f-53fd-4cf7-8a56-e2c6f0b0de7b`  

**Excerpt:**
```text
Visit Information: - Patient ID: 1000000110 - Encounter ID: 481878 - Visit Date: 11/13/2008  Subjective: This is a 36-year-old Hispanic male, fluent in German, who is presenting for a follow-up visit. He has been previously diagnosed with multiple chronic conditions, primarily asthma, which he manages with inhaled corticosteroids. He reports that his asthma symptoms, including intermittent wheezing and shortness of breath, are predominantly triggered by exposure to pollen and dust. There have been no recent severe exacerbations requiring hospitalization.   In addition to asthma, the patient is monitoring several coexisting conditions. He has a history of both rheumatoid arthritis and polymyositis, which contribute to generalized fatigue and periodic limb pain, respectively. These symptoms have been relatively stable with his current medication regimen including Prednisone. He also reports occasional tingling in his extremities secondary to mononeuritis, but no recent exacerbations.  Fo...
```

**✅ Total MMR Results Returned:** `5`

## 4. Generating Structured Responses with an LLM (RAG Retrieval)

In this final step, we use a Large Language Model (LLM) to analyze the clinical notes retrieved in the previous section. By injecting these relevant notes into a structured prompt, we enable the LLM to generate clinically useful, structured responses.

This completes the Retrieval-Augmented Generation (RAG) pipeline, connecting search results to intelligent language generation.

### Key Steps

1. **Create a Prompt Template (Step 4.1)**
   - Defines a reusable prompt structure that includes placeholders for both the query and the retrieved clinical context.
   - Specifies a structured output format, including metadata fields such as patient ID, encounter date, and a clinical summary.

2. **Invoke the LLM with Retrieved Context (Step 4.2)**
   - Fills the prompt with retrieved notes and the clinical query.
   - Sends the prompt to a local LLM (e.g., Qwen2 or LLaMA 3 via Ollama).
   - Returns a structured, context-aware response to the clinical question.

### Why It Matters

This generation step demonstrates the power of combining semantic search with generative AI:
- Produces context-rich answers grounded in patient data
- Supports clinical summarization and decision support
- Enables zero-shot reasoning over real-world clinical notes

<img src="./images/rag_generation.png" alt="RAG Generation" width="1250">


In [45]:
# -----------------------------------------------------------
# 4.1. Create a Prompt Template for LLM Querying
# -----------------------------------------------------------
# This prompt template guides the LLM to generate structured summaries
# from clinical notes retrieved via similarity search.

# It includes placeholders for:
# - {retrieved_docs}: Injects the top-matching clinical notes
# - {query}: A user-defined clinical question

# Output Expectations:
# - One structured response per patient
# - Includes metadata for traceability
# - Summarizes and answers the query based on each patient's most recent note

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "You are a medical assistant analyzing clinical notes. Based on the following records:\n\n"
    "{retrieved_docs}\n\n"
    "Answer the question: {query} using the following structure:\n"
    "   - Patient Num: <value>, Gender: <value>, Age: <value>, Race: <value>\n"
    "   - Visit Date: <value>\n"
    "   - Summary: One paragraph summarizing the patient note and answering the question.\n\n"
    "   - Has Asthma: <Yes/No>"
    "Instructions:\n"
    "- Show all patients that are relevant to the query.\n"
    "- Only consider the most recent note for each patient (identified by patient_num)."
)

display(prompt_template)


PromptTemplate(input_variables=['query', 'retrieved_docs'], input_types={}, partial_variables={}, template='You are a medical assistant analyzing clinical notes. Based on the following records:\n\n{retrieved_docs}\n\nAnswer the question: {query} using the following structure:\n   - Patient Num: <value>, Gender: <value>, Age: <value>, Race: <value>\n   - Visit Date: <value>\n   - Summary: One paragraph summarizing the patient note and answering the question.\n\n   - Has Asthma: <Yes/No>Instructions:\n- Show all patients that are relevant to the query.\n- Only consider the most recent note for each patient (identified by patient_num).')

In [46]:
# -----------------------------------------------------------
# 4.2. Use Retrieved Context to Invoke LLM and Generate Response
# -----------------------------------------------------------
# This step completes the RAG workflow by injecting the retrieved clinical notes
# into a structured prompt and using an LLM (via Ollama) to generate a response.

from langchain_ollama import ChatOllama
from IPython.display import display, Markdown

# Initialize the local Ollama model (e.g., Qwen 2, LLaMA 3, etc.)
model = ChatOllama(model="llama3:70b")

# Prepare the context text (combine page_content from results list)
retrieved_context = "\n\n---\n\n".join([doc.page_content for doc in results])

# Fill the prompt template with retrieved notes and the user's query
final_prompt = prompt_template.format(
    retrieved_docs=retrieved_context,
    query=query
)

# Generate a structured response using the LLM
response = model.invoke(final_prompt)

# Display the LLM-generated output
display(Markdown("### LLM-Generated Response"))
display(Markdown(response.content))


### LLM-Generated Response

Here are the results:

**Patient 1**
- Patient Num: 1000000005, Gender: Female, Age: 32, Race: Hispanic
- Visit Date: June 21, 2005
- Summary: The patient presents with continued asthma symptoms, including shortness of breath, wheezing, and cough. She is taking albuterol inhaler for rescue and daily fluticasone inhaler.
- Has Asthma: Yes

**Patient 2**
- Patient Num: 1000000088, Gender: Male, Age: 9, Race: Asian
- Visit Date: Oct 28, 2004
- Summary: The patient has a history of asthma and allergic rhinitis. He is taking daily inhaled corticosteroids for asthma and albuterol for acute symptoms.
- Has Asthma: Yes

**Patient 3**
- Patient Num: 1000000112, Gender: Male, Age: 12, Race: Black
- Visit Date: June 10, 2005
- Summary: The patient has chronic asthma with subtypes including acute exacerbations and status asthmaticus. He is taking Atrovent (ipratropium bromide) and Flovent (fluticasone).
- Has Asthma: Yes

**Patient 4**
- Patient Num: 1000000110, Gender: Male, Age: 36, Race: Hispanic
- Visit Date: November 13, 2008
- Summary: The patient has asthma managed with inhaled corticosteroids. He is taking no specific medication for asthma.
- Has Asthma: Yes